In [2]:
import pandas as pd
import numpy as np

/usr/lib64/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# CIDACS: SUPORTE EM ROTINAS DE PRODUÇÃO DE DADOS - Data analysis / Avaliation

This is a practical and technical test whose objective is to evaluate your data analysis and interpretation. 

The dataset is related to the Basic Health Unity (UBS). It was made available by the Brazilian Ministry of Health and it shows the cities that received the government program.



## Challenge
Based on the data provided, answer the following questions:

1. How many cities have at least one UBS?
2. How many UBS in Vitoria received well above average equipment performance reviews?
3. How many UBS received only well above average performances reviews?
3. How many UBS received only the worse possible performance reviews?
    1. Print all the cities that have UBS in this situation

## Data dictionary


* cod_munic `IBGE code , lenght: 7 characters.` 
* dsc_bairro `Neighborhood name.`
* dsc_cidade `City name.`
* cod_cnes `CNES code`
* nom_estab `Establishment name`
* dsc_endereco `UBS address`
* dsc_estrut_fisic_ambiencia `Situation related to general structure`
* dsc_adap_defic_fisic_idosos `Situation related to adaptations for the disabled and elderly`
* dsc_equipamentos `Situation related to equipament`
* dsc_medicamentos `Situation related to medicines`


In [32]:
df = pd.read_csv('./ubs_avaliation_npd.csv')

In [33]:
len(df)

37690

In [37]:
df.head()

,Unnamed: 0,nom_estab,dsc_endereco,dsc_bairro,dsc_cidade,dsc_telefone,dsc_estrut_fisic_ambiencia,dsc_adap_defic_fisic_idosos,dsc_equipamentos,dsc_medicamentos
0,0,US OSWALDO DE SOUZA,TV ADALTO BOTELHO,GETULIO VARGAS,Aracaju,7931791326,Good,Excellent,Average,Good
1,1,USF ENFERMEIRO PEDRO JACINTO AREA 09,R 15 DE AGOSTO,CENTRO,Rio Largo,Nao se aplica,Average,Average,Average,Average
2,2,UNIDADE DE ATENCAO PRIMARIA SAUDE DA FAMILIA,RUA GUILHERME BRUXEL,CENTRO,Perobal,4436251462,Excellent,Excellent,Average,Excellent
3,3,POSTO DE SAUDE DE BOM JESUS DA ALDEIA,RUA TEOFILO OTONI,ALDEIA,Jequitinhonha,3337411423,Average,Good,Average,Average
4,4,POSTO ANCORA URUBA,RODOVIA PB N 065,SITIO,Mataraca,Nao se aplica,Good,Good,Average,Average


### How many cities have at least one UBS?

In [23]:
len(df.groupby('dsc_cidade', as_index=False))

5226

### How many UBS in Vitória received well above average equipment performance reviews?

In [36]:
value = 'Excellent'
df_filtered = df[ (df.dsc_cidade == 'Vitoria' ) & (df.dsc_equipamentos == value) ]
len(df_filtered)

13

### How many UBS received only well above average performances reviews?

In [57]:
value = 'Desempenho muito acima da média'
df_filtered = df[ (df.dsc_estrut_fisic_ambiencia == value) & (df.dsc_adap_defic_fisic_idosos == value) & (df.dsc_equipamentos == value) & (df.dsc_medicamentos == value)]
#len(df_filtered.groupby('dsc_endereco', as_index=False)) aqui você não conta uma UBS que está no mesmo endereço que outra.
len(df_filtered)

102

### How many UBS received only the worse possible performance reviews?

In [39]:
value = 'Average'
df_filtred = df[ (df.dsc_estrut_fisic_ambiencia == value) & (df.dsc_adap_defic_fisic_idosos == value) & (df.dsc_equipamentos == value) & (df.dsc_medicamentos == value)]
len(df_filtred.groupby('dsc_endereco', as_index=False))

8588

In [40]:
value = 'Average' #Aqui pode dar um pouco de problema pelo espaço duplo entre duas palavras.
df_filtered = df[ (df.dsc_estrut_fisic_ambiencia == value) & (df.dsc_adap_defic_fisic_idosos == value) & (df.dsc_equipamentos == value) & (df.dsc_medicamentos == value)]
print(len(df_filtered.groupby('dsc_endereco', as_index=False)))
df_filtered.groupby('dsc_cidade', as_index=False).agg('count')['dsc_cidade']

8588


0                              Abadiania
1                             Abaetetuba
2                        Abel Figueiredo
3                           Abreu e Lima
4                             Acailandia
5                              Acajutiba
6                                  Acara
7                                Acarape
8                                 Acarau
9                                  Acari
10                              Acorizal
11                            Acrelandia
12                               Acreuna
13                          Adrianopolis
14                              Adustina
15                 Afogados da Ingazeira
16                        Afonso Claudio
17                          Afonso Cunha
18                          Agricolandia
19                    Agua Azul do Norte
20                              Agua Boa
21                           Agua Branca
22                            Agua Clara
23                 Agua Doce do Maranhao
24              

In [25]:
cols = df.select_dtypes(include=[np.object]).columns
df = df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))


In [26]:
columns = ['dsc_estrut_fisic_ambiencia','dsc_adap_defic_fisic_idosos','dsc_equipamentos','dsc_medicamentos']
dic = {
    'Desempenho acima da media':'Good',
    'Desempenho mediano ou um pouco abaixo da media':'Average',
    'Desempenho muito acima da media':'Excellent'
}
def change_categories(line):
    try:
        return dic[line]
    except:
        return 'Average'
for column in columns:
    df[column] = df[column].apply(change_categories)